# Sources
* BiLSTM model architecture based on [Ozols et. al., 2021](https://www.mdpi.com/1422-0067/22/6/3071/htm)
* Co-teaching+ loss function and training process adaptations are based on [Yu et al., 2019](https://arxiv.org/abs/1901.04215), and official implementation on [Github](https://github.com/xingruiyu/coteaching_plus)


In [1]:
import os
import sys
import csv
import pickle
import random
import math
import numpy as np
from time import time
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import (
    pack_padded_sequence,
    pad_packed_sequence
)
from torch.utils.data import (
    Dataset,
    DataLoader,
    Sampler,
    BatchSampler
)

from tokenizers import BertWordPieceTokenizer

from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [2]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(1234)

In [3]:
def read_data(path):
    with open(path, "r") as csvfile:
        train_data = list(csv.reader(csvfile))[1:]  # skip col name
        sents, lbls = [], []
        for s, l in train_data:
            sents.append(s)
            lbls.append(l)
    return sents, lbls


def apply_random_masking(seq, num_tokens):
    """
    Mask `num_tokens` as 1 (i.e. [UNK]) at random positions per sequence.
    """
    dist = torch.rand(seq.shape)
    m, _ = torch.topk(dist, num_tokens)
    return seq * (dist < m) + (dist == m) * 1


def regularized_auc(train_auc, dev_auc, threshold=0.0025):
    """
    Returns development AUC if overfitting is below threshold, otherwise 0.
    """
    return dev_auc if (train_auc - dev_auc) < threshold else 0


def gelu(x):
    """
    Facebook Research implementation of the gelu activation function.
    
    For information: OpenAI GPT's gelu is slightly different
    (and gives slightly different results):
    0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
    """
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))


def trainable_model_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def total_model_params(model):
    return sum(p.numel() for p in model.parameters())

In [4]:
class CleavageDataset(Dataset):
    def __init__(self, seq, lbl):
        self.seq = seq
        self.lbl = lbl

    def __getitem__(self, idx):
        return self.seq[idx], self.lbl[idx]

    def __len__(self):
        return len(self.lbl)


class BucketSampler(Sampler):
    def __init__(self, seqs, batch_size):

        # pair each sequence with their *tokenized* length
        indices = [(idx, len(tokenizer.encode(s).ids)) for idx, s in enumerate(seqs)]
        random.shuffle(indices)

        idx_pools = []
        # generate pseudo-random batches of (arbitrary) size batch_size * 100
        # each batch of size batch_size * 100 is sorted in itself by seq length
        for i in range(0, len(indices), batch_size * 100):
            idx_pools.extend(
                sorted(indices[i : i + batch_size * 100], key=lambda x: x[1])
            )

        # filter only indices
        self.idx_pools = [x[0] for x in idx_pools]

    def __iter__(self):
        return iter(self.idx_pools)

    def __len__(self):
        return len(self.idx_pools)


class TrainBatch:
    def __init__(self, batch):
        ordered_batch = list(zip(*batch))
        seq = torch.tensor(
            [s.ids for s in tokenizer.encode_batch(ordered_batch[0])], dtype=torch.int64
        )
        self.seq = apply_random_masking(seq, num_tokens=1)
        self.lbl = torch.tensor([int(l) for l in ordered_batch[1]], dtype=torch.float)
        self.lengths = torch.tensor([self.seq.shape[1]] * self.seq.shape[0], dtype=torch.int64)

    def pin_memory(self):
        self.seq = self.seq.pin_memory()
        self.lbl = self.lbl.pin_memory()
        return self


def train_wrapper(batch):
    return TrainBatch(batch)


class EvalBatch:
    def __init__(self, batch):
        ordered_batch = list(zip(*batch))
        self.seq = torch.tensor(
            [s.ids for s in tokenizer.encode_batch(ordered_batch[0])], dtype=torch.int64
        )
        self.lbl = torch.tensor([int(l) for l in ordered_batch[1]], dtype=torch.float)
        self.lengths = torch.tensor([self.seq.shape[1]] * self.seq.shape[0], dtype=torch.int64)

    def pin_memory(self):
        self.seq = self.seq.pin_memory()
        self.lbl = self.lbl.pin_memory()
        return self


def eval_wrapper(batch):
    return EvalBatch(batch)

In [5]:
class BiLSTM(nn.Module):
    def __init__(
        self,
        vocab_size,
        embedding_dim,
        rnn_size1,
        rnn_size2,
        hidden_size,
        dropout,
    ):
        super().__init__()

        self.embedding = nn.Embedding(
            num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=1
        )

        self.dropout = nn.Dropout(dropout)

        self.lstm1 = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=rnn_size1,
            bidirectional=True,
            batch_first=True,
        )

        self.lstm2 = nn.LSTM(
            input_size=2 * rnn_size1,
            hidden_size=rnn_size2,
            bidirectional=True,
            batch_first=True,
        )

        self.fc1 = nn.Linear(rnn_size2 * 2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, seq, lengths):
        # input shape: (batch_size, seq_len=10)
        embedded = self.dropout(self.embedding(seq))

        packed_embeddings = pack_padded_sequence(
            embedded, lengths, batch_first=True, enforce_sorted=False
        )

        # input shape: (batch_size, seq_len, embedding_dim)
        out, _ = self.lstm1(packed_embeddings)

        # input shape: (batch_size, seq_len, 2*rnn_size1)
        out, _ = self.lstm2(out)

        unpacked_output, _ = pad_packed_sequence(out, batch_first=True, padding_value=1)

        # input shape: (batch_size, seq_len, 2*hidden_size)
        pooled, _ = torch.max(unpacked_output, dim=1)

        # input shape; (batch_size, 2*hidden_size)
        out = self.dropout(gelu(self.fc1(pooled)))

        # input shape: (batch_size, hidden_size)
        # output shape: (batch_size)
        return self.fc2(out).squeeze()

In [6]:
def loss_coteaching(y_1, y_2, t, forget_rate):
    criterion = nn.BCEWithLogitsLoss(reduction='none')
    
    loss_1 = criterion(y_1, t)
    ind_1_sorted = np.argsort(loss_1.data.cpu())
    loss_1_sorted = loss_1[ind_1_sorted]

    loss_2 = criterion(y_2, t)
    ind_2_sorted = np.argsort(loss_2.data.cpu())
    loss_2_sorted = loss_2[ind_2_sorted]

    remember_rate = 1 - forget_rate
    num_remember = int(remember_rate * len(loss_1_sorted))

    ind_1_update = ind_1_sorted[:num_remember]
    ind_2_update = ind_2_sorted[:num_remember]
    
    # exchange
    loss_1_update = criterion(y_1[ind_2_update], t[ind_2_update])
    loss_2_update = criterion(y_2[ind_1_update], t[ind_1_update])

    return torch.sum(loss_1_update)/num_remember, torch.sum(loss_2_update)/num_remember

In [7]:
def train(model1, optim1, model2, optim2, loader, forget_rate):
    epoch_loss1, num_correct1, total = 0, 0, 0
    epoch_loss2, num_correct2 = 0, 0
    preds1, preds2, lbls = [], [], []

    for batch in tqdm(loader, desc="Train: ", file=sys.stdout, unit="batches"):
        seq, lbl, lengths = batch.seq, batch.lbl, batch.lengths
        seq, lbl = seq.to(device), lbl.to(device)

        scores1 = model1(seq, lengths)
        pred1 = scores1 > 0
        scores2 = model2(seq, lengths)
        pred2 = scores2 > 0

        inds = torch.where(pred1 != pred2)
        if len(inds[0]) * (1 - forget_rate) < 1:
            loss1 = criterion(scores1, lbl)
            loss2 = criterion(scores2, lbl)
        else:
            loss1, loss2 = loss_coteaching(scores1, scores2, lbl, forget_rate)

        optim1.zero_grad()
        loss1.backward()
        optim1.step()
        
        optim2.zero_grad()
        loss2.backward()
        optim2.step()

        epoch_loss1 += loss1.item()
        epoch_loss2 += loss2.item()
        num_correct1 += (pred1 == lbl).sum().item()
        num_correct2 += (pred2 == lbl).sum().item()
        total += seq.shape[0]
        preds1.extend(scores1.detach().tolist())
        preds2.extend(scores2.detach().tolist())
        lbls.extend(lbl.detach().tolist())
        
    return (
        epoch_loss1 / total,
        epoch_loss2 / total,
        num_correct1 / total,
        num_correct2 / total,
        roc_auc_score(lbls, preds1),
        roc_auc_score(lbls, preds2),
    )

In [8]:
def evaluate(model1, model2, loader, criterion):
    epoch_loss1, num_correct1, total = 0, 0, 0
    epoch_loss2, num_correct2 = 0, 0
    preds1, preds2, lbls = [], [], []
    
    for batch in tqdm(loader, desc="Eval: ", file=sys.stdout, unit="batches"):
        seq, lbl, lengths = batch.seq, batch.lbl, batch.lengths
        seq, lbl = seq.to(device), lbl.to(device)
        
        scores1 = model1(seq, lengths)
        scores2 = model2(seq, lengths)

        loss1 = criterion(scores1, lbl)
        loss2 = criterion(scores2, lbl)
        
        epoch_loss1 += loss1.item()
        epoch_loss2 += loss2.item()
        num_correct1 += ((scores1 > 0) == lbl).sum().item()
        num_correct2 += ((scores2 > 0) == lbl).sum().item()
        total += seq.shape[0]
        preds1.extend(scores1.detach().tolist())
        preds2.extend(scores2.detach().tolist())
        lbls.extend(lbl.detach().tolist())
        
    return (
        epoch_loss1 / total,
        epoch_loss2 / total,
        num_correct1 / total,
        num_correct2 / total,
        roc_auc_score(lbls, preds1),
        roc_auc_score(lbls, preds2)
    )

In [9]:
def test(model, loader, criterion):
    epoch_loss, num_correct, total = 0, 0, 0
    preds, lbls = [], []
    
    for batch in tqdm(loader, desc="Eval: ", file=sys.stdout, unit="batches"):
        seq, lbl, lengths = batch.seq, batch.lbl, batch.lengths
        seq, lbl = seq.to(device), lbl.to(device)
        
        scores = model(seq, lengths)

        loss = criterion(scores, lbl)
        
        epoch_loss += loss.item()
        num_correct += ((scores > 0) == lbl).sum().item()
        total += seq.shape[0]
        preds.extend(scores.detach().tolist())
        lbls.extend(lbl.detach().tolist())
        
    return epoch_loss / total, num_correct / total, roc_auc_score(lbls, preds)

In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
vocab_file = "../../params/n_term/bwp_params/50k/vocab.txt"

# tokenizer serves as vocab at the same time
tokenizer = BertWordPieceTokenizer.from_file(vocab_file)
tokenizer.enable_padding(pad_token="[PAD]")

# load train and dev data
train_seqs, train_lbl = read_data('../../data/n_train.csv')
dev_seqs, dev_lbl = read_data('../../data/n_val.csv')

In [11]:
NUM_EPOCHS = 15
BATCH_SIZE = 512
VOCAB_SIZE = tokenizer.get_vocab_size()
EMBEDDING_DIM = 150
RNN_SIZE1 = 256
RNN_SIZE2 = 512
HIDDEN_SIZE = 128
DROPOUT = 0.5
LEARNING_RATE = 1e-4

NUM_GRADUAL = 10 # how many epochs for linear drop rate
NOISY_RATE = 0.2
FORGET_RATE = 0.2
EXPONENT = 1

# define drop rate schedule
rate_schedule = np.ones(NUM_EPOCHS)*FORGET_RATE
rate_schedule[:NUM_GRADUAL] = np.linspace(0, FORGET_RATE**EXPONENT, NUM_GRADUAL)

params = {
    "vocab_size": VOCAB_SIZE,
    "embedding_dim": EMBEDDING_DIM,
    "rnn_size1": RNN_SIZE1,
    "rnn_size2": RNN_SIZE2,
    "hidden_size": HIDDEN_SIZE,
    "dropout": DROPOUT
}

model1 = BiLSTM(**params).to(device)
model2 = BiLSTM(**params).to(device)

optimizer1 = optim.Adam(model1.parameters(), lr=LEARNING_RATE)
optimizer2 = optim.Adam(model2.parameters(), lr=LEARNING_RATE)
criterion = nn.BCEWithLogitsLoss()

# create train and dev loader
train_data = CleavageDataset(train_seqs, train_lbl)
train_loader = DataLoader(train_data, batch_size = BATCH_SIZE, shuffle=True, collate_fn=train_wrapper, pin_memory=True, num_workers=10)

dev_data = CleavageDataset(dev_seqs, dev_lbl)
dev_loader = DataLoader(dev_data, batch_size = BATCH_SIZE, shuffle=True, collate_fn=eval_wrapper, pin_memory=True, num_workers=10)

In [12]:
start = time()
print("Starting Training.")
highest_val_auc = 0

# normal training loop
for epoch in range(1, NUM_EPOCHS + 1):
    model1.train()
    model2.train()
    train_loss1, train_loss2, train_acc1, train_acc2, train_auc1, train_auc2 = train(
        model1=model1,
        optim1=optimizer1,
        model2=model2,
        optim2=optimizer2,
        loader=train_loader,
        forget_rate=rate_schedule[epoch-1],
    )

    model1.eval()
    model2.eval()
    with torch.no_grad():
        val_loss1, val_loss2, val_acc1, val_acc2, val_auc1, val_auc2 = evaluate(
            model1=model1,
            model2=model2,
            loader=dev_loader,
            criterion=criterion
        )
    
    print(
        f"Model 1 Training:   [Epoch {epoch:2d}, Loss: {train_loss1:8.6f}, Acc: {train_acc1:.4f}, AUC: {train_auc1:.4f}]"
    )
    print(
        f"Model 1 Evaluation: [Epoch {epoch:2d}, Loss: {val_loss1:8.6f}, Acc: {val_acc1:.4f}, AUC: {val_auc1:.4f}]"
    )
    print(
        f"Model 2 Training:   [Epoch {epoch:2d}, Loss: {train_loss2:8.6f}, Acc: {train_acc2:.4f}, AUC: {train_auc2:.4f}]"
    )
    print(
        f"Model 2 Evaluation: [Epoch {epoch:2d}, Loss: {val_loss2:8.6f}, Acc: {val_acc2:.4f}, AUC: {val_auc2:.4f}]"
    )

    if val_auc1 > val_auc2:
        reg_auc = regularized_auc(train_auc1, val_auc1, threshold=0)
        model = model1.state_dict()
        print('saved model1')
    else:
        reg_auc = regularized_auc(train_auc2, val_auc2, threshold=0)
        model = model2.state_dict()
        print('saved model2')
    if reg_auc > highest_val_auc:
        highest_val_auc = reg_auc
        path = f"../../params/n_term/bwpBiLSTM_coteaching_plus/auc{reg_auc:.4f}_epoch{epoch}.pt"
        torch.save(model, path)

print("Finished Training.")
train_time = (time() - start) / 60
print(f"Training took {train_time} minutes.")

Starting Training.
Eval: 100%|██████████████████████████████████████████████████████| 281/281 [00:01<00:00, 177.72batches/s]
Model 1 Training:   [Epoch  1, Loss: 0.000928, Acc: 0.8183, AUC: 0.5317]
Model 1 Evaluation: [Epoch  1, Loss: 0.000919, Acc: 0.8179, AUC: 0.5737]
Model 2 Training:   [Epoch  1, Loss: 0.000929, Acc: 0.8169, AUC: 0.5306]
Model 2 Evaluation: [Epoch  1, Loss: 0.000920, Acc: 0.8179, AUC: 0.5711]
saved model1
Eval: 100%|██████████████████████████████████████████████████████| 281/281 [00:01<00:00, 168.20batches/s]
Model 1 Training:   [Epoch  2, Loss: 0.000919, Acc: 0.8183, AUC: 0.5586]
Model 1 Evaluation: [Epoch  2, Loss: 0.000914, Acc: 0.8179, AUC: 0.5914]
Model 2 Training:   [Epoch  2, Loss: 0.000920, Acc: 0.8183, AUC: 0.5540]
Model 2 Evaluation: [Epoch  2, Loss: 0.000915, Acc: 0.8179, AUC: 0.5897]
saved model1
Eval: 100%|██████████████████████████████████████████████████████| 281/281 [00:01<00:00, 172.65batches/s]
Model 1 Training:   [Epoch  3, Loss: 0.000914, Acc: 0

In [13]:
test_path = '../../data/n_test.csv'
test_seqs, test_lbls = read_data(test_path)

test_data = CleavageDataset(test_seqs, test_lbls)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, collate_fn=eval_wrapper, pin_memory=True, num_workers=10)

# load best model, evaluate on test set
best_model = sorted(
    [f for f in os.listdir("../../params/n_term/bwpBiLSTM_coteaching_plus/") if f.endswith(".pt")],
    reverse=True,
)[0]
print("Loaded model: ", best_model)
model1.load_state_dict(torch.load('../../params/n_term/bwpBiLSTM_coteaching_plus/' + best_model))
model1.eval()
with torch.no_grad():
    test_loss, test_acc, test_auc = test(model1, test_loader, criterion)
print(
    f"Test Set Performance: Loss: {test_loss:.6f}, Acc: {test_acc:.4f}, AUC: {test_auc:.4f}"
)
print(
    f"Total model params: {total_model_params(model1)}, trainable model params: {trainable_model_params(model1)}"
)

Loaded model:  auc0.6812_epoch15.pt
Eval: 100%|██████████████████████████████████████████████████████| 281/281 [00:01<00:00, 271.81batches/s]
Test Set Performance: Loss: 0.000925, Acc: 0.8197, AUC: 0.6844
Total model params: 12669409, trainable model params: 12669409
